In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install deepctr[gpu]

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.2/141.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting unins

In [6]:
#train
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models import MMOE,PLE

data = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_train_subgroup.csv').drop(columns=['user_id','subgroups'])
feature_names =data.columns.tolist()
new_feature_list=[]
for i in range(len(feature_names)):
    new_feature = str(i)+"_feature"
    new_feature_list.append(new_feature)
feature_names = new_feature_list
data.columns = feature_names
seen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_seen_subgroup.csv').drop(columns=['user_id','idx'])
seen_test.columns = feature_names
unseen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_unseen_subgroup.csv').drop(columns=['user_id','idx'])
unseen_test.columns = feature_names
sparse_features = feature_names
fixlen_feature_columns = [SparseFeat(feat, data[feat].max() + 1, embedding_dim=4) for feat in sparse_features]
data = data.dropna()
dnn_feature_columns = fixlen_feature_columns

feature_names = sparse_features
train_model_input = {name: data[name] for name in feature_names}
test_seen_model_input = {name: seen_test[name] for name in feature_names}
test_unseen_model_input = {name: unseen_test[name] for name in feature_names}
subgroup_info = pd.read_csv("./drive/MyDrive/data/subgroup/subgroups.csv")
label_list = subgroup_info['subgroup_name'].tolist()

new_label = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_train_subgroup_label_multi_label.csv')
label_name = new_label.columns.tolist()
label_col = new_label.columns
label = []
for name in label_col:
    l = new_label[name].values
    label.append(l)



# 4.Define Model,train,predict and evaluate
model = PLE(dnn_feature_columns, tower_dnn_hidden_units=[], task_types=['binary']*len(label_col),
            task_names = label_col)
model.compile("adam", loss=["binary_crossentropy"]*len(label_col),
              metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, label,
                    batch_size=1024, epochs=25, verbose=2)

pred_ans = model.predict(test_seen_model_input, batch_size=256)
seen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_seen_subgroup.csv',usecols=['user_id'])
for idx,label in enumerate(label_name):
    seen_test[label] = pred_ans[idx]
seen_test.to_csv('./drive/MyDrive/data/subgroup/mmoe_pred_test_seen_subgroup.csv',index=False)

pred_ans = model.predict(test_unseen_model_input, batch_size=256)
unseen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_unseen_subgroup.csv',usecols=['user_id'])
for idx,label in enumerate(label_name):
    unseen_test[label] = pred_ans[idx]
unseen_test.to_csv('./drive/MyDrive/data/subgroup/mmoe_pred_test_unseen_subgroup.csv',index=False)
save_model(model, './drive/MyDrive/data/ckpt/PLE_subgroup.h5')

Epoch 1/25
58/58 - 91s - loss: 24.2095 - 1_loss: 0.4876 - 2_loss: 0.1669 - 3_loss: 0.5098 - 4_loss: 0.3577 - 5_loss: 0.4050 - 6_loss: 0.4393 - 7_loss: 0.5228 - 8_loss: 0.3899 - 9_loss: 0.1834 - 10_loss: 0.1542 - 11_loss: 0.1469 - 12_loss: 0.1621 - 13_loss: 0.2982 - 14_loss: 0.1926 - 15_loss: 0.2583 - 16_loss: 0.1742 - 17_loss: 0.1539 - 18_loss: 0.1600 - 19_loss: 0.2792 - 20_loss: 0.1699 - 21_loss: 0.1827 - 22_loss: 0.1810 - 23_loss: 0.4257 - 24_loss: 0.2075 - 25_loss: 0.2365 - 26_loss: 0.1616 - 27_loss: 0.1729 - 28_loss: 0.1953 - 29_loss: 0.4398 - 30_loss: 0.1815 - 31_loss: 0.1532 - 32_loss: 0.2798 - 33_loss: 0.3742 - 34_loss: 0.3584 - 35_loss: 0.2694 - 36_loss: 0.4533 - 37_loss: 0.3789 - 38_loss: 0.3155 - 39_loss: 0.3271 - 40_loss: 0.4565 - 41_loss: 0.1850 - 42_loss: 0.1642 - 43_loss: 0.1632 - 44_loss: 0.1576 - 45_loss: 0.1794 - 46_loss: 0.1805 - 47_loss: 0.1700 - 48_loss: 0.1601 - 49_loss: 0.1919 - 50_loss: 0.4895 - 51_loss: 0.5823 - 52_loss: 0.2010 - 53_loss: 0.2513 - 54_loss: 0.200

In [ ]:
#test
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr.feature_column import SparseFeat, DenseFeat, get_feature_names
from deepctr.models import MMOE,PLE


seen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_seen_subgroup.csv').drop(columns=['user_id','idx'])
feature_names =seen_test.columns.tolist()
new_feature_list=[]
for i in range(len(feature_names)):
    new_feature = str(i)+"_feature"
    new_feature_list.append(new_feature)
feature_names = new_feature_list

seen_test.columns = feature_names
unseen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_unseen_subgroup.csv').drop(columns=['user_id','idx'])
unseen_test.columns = feature_names

sparse_features = feature_names
fixlen_feature_columns = [SparseFeat(feat, data[feat].max() + 1, embedding_dim=4) for feat in sparse_features]
dnn_feature_columns = fixlen_feature_columns

feature_names = sparse_features

test_seen_model_input = {name: seen_test[name] for name in feature_names}
test_unseen_model_input = {name: unseen_test[name] for name in feature_names}



model = load_model('./drive/MyDrive/data/PLE.h5',custom_objects)# load_model,just add a parameter
pred_ans = model.predict(test_seen_model_input, batch_size=256)
seen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_seen_subgroup.csv',usecols=['user_id'])
for idx,label in enumerate(label_name):
    seen_test[label] = pred_ans[idx]
seen_test.to_csv('./drive/MyDrive/data/subgroup/mmoe_pred_test_seen_subgroup.csv',index=False)

pred_ans = model.predict(test_unseen_model_input, batch_size=256)
unseen_test = pd.read_csv('./drive/MyDrive/data/subgroup/mmoe_test_unseen_subgroup.csv',usecols=['user_id'])
for idx,label in enumerate(label_name):
    unseen_test[label] = pred_ans[idx]
unseen_test.to_csv('./drive/MyDrive/data/subgroup/mmoe_pred_test_unseen_subgroup.csv',index=False)